# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,birzai,56.2000,24.7500,35.87,86,6,8.70,LT,1697499774
1,1,port alfred,-33.5906,26.8910,60.19,94,85,8.75,ZA,1697499774
2,2,yellowknife,62.4560,-114.3525,45.41,94,75,8.05,CA,1697499775
3,3,arraial do cabo,-22.9661,-42.0278,73.36,88,75,18.41,BR,1697499775
4,4,port-aux-francais,-49.3500,70.2167,37.06,90,93,28.45,TF,1697499775


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,birzai,56.2000,24.7500,35.87,86,6,8.70,LT,1697499774
1,1,port alfred,-33.5906,26.8910,60.19,94,85,8.75,ZA,1697499774
2,2,yellowknife,62.4560,-114.3525,45.41,94,75,8.05,CA,1697499775
3,3,arraial do cabo,-22.9661,-42.0278,73.36,88,75,18.41,BR,1697499775
4,4,port-aux-francais,-49.3500,70.2167,37.06,90,93,28.45,TF,1697499775


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
fit_criteria_city = city_data_df.loc[(city_data_df["Max Temp"] < 35) 
                                     & (city_data_df["Max Temp"] > 20) 
                                     & (city_data_df["Cloudiness"] < 21) 
                                     & (city_data_df["Wind Speed"] <10)]
                                    
fit_criteria_city.dropna()
    
fit_criteria_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
103,103,longyearbyen,78.2186,15.6401,22.84,63,20,9.22,SJ,1697499499
116,116,zeya,53.7500,127.2667,28.49,52,14,1.34,RU,1697499799
128,128,stanley,54.8680,-1.6985,33.78,100,0,2.30,GB,1697499802
261,261,altay,47.8667,88.1167,33.55,46,7,8.30,CN,1697499831
437,437,tromso,69.6496,18.9570,33.82,80,20,3.44,NO,1697499871
470,470,bulgan,48.8125,103.5347,32.94,57,0,4.70,MN,1697499878
537,537,holmsund,63.7060,20.3641,33.94,86,0,8.05,SE,1697499893


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
hotel_df = fit_criteria_city[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
103,longyearbyen,SJ,78.2186,15.6401,63,
116,zeya,RU,53.7500,127.2667,52,
128,stanley,GB,54.8680,-1.6985,100,
261,altay,CN,47.8667,88.1167,46,
437,tromso,NO,69.6496,18.9570,80,
470,bulgan,MN,48.8125,103.5347,57,
537,holmsund,SE,63.7060,20.3641,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
longitude = hotel_df["Lng"]
latitude = hotel_df["Lat"]
filters = f"circle:{longitude}, {latitude}, {radius}"
bias = f"proximity:{longitude}, {latitude}"
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "radius": radius,
    "filter": filters,
    "bias": bias   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filters"] = filters
    params["bias"] = bias
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
longyearbyen - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
zeya - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
stanley - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
altay - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
tromso - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
bulgan - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not mat

,City,Country,Lat,Lng,Humidity,Hotel Name
103,longyearbyen,SJ,78.2186,15.6401,63,No hotel found
116,zeya,RU,53.7500,127.2667,52,No hotel found
128,stanley,GB,54.8680,-1.6985,100,No hotel found
261,altay,CN,47.8667,88.1167,46,No hotel found
437,tromso,NO,69.6496,18.9570,80,No hotel found
470,bulgan,MN,48.8125,103.5347,57,No hotel found
537,holmsund,SE,63.7060,20.3641,86,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)